In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../data/MEX/")[0:100]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

100 municipalities.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .001)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .70,
                         convergence_dims = (224, 224),
                         plot = False, 
                         v = False)

In [5]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [6]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  60
Num validation:  40


In [10]:
image_names

['../data/MEX/484001008/pngs/484001008_2010_all_box484001008_MAY.png',
 '../data/MEX/484004004/pngs/484004004_2010_all_box484004004_MAY.png',
 '../data/MEX/484004001/pngs/484004001_2010_all_box484004001_MAY.png',
 '../data/MEX/484005010/pngs/484005010_2010_all_box484005010_MAY.png',
 '../data/MEX/484001005/pngs/484001005_2010_all_box484001005_MAY.png',
 '../data/MEX/484001007/pngs/484001007_2010_all_box484001007_MAY.png',
 '../data/MEX/484001006/pngs/484001006_2010_all_box484001006_MAY.png',
 '../data/MEX/484001001/pngs/484001001_2010_all_box484001001_MAY.png',
 '../data/MEX/484004005/pngs/484004005_2010_all_box484004005_MAY.png',
 '../data/MEX/484005001/pngs/484005001_2010_all_box484005001_MAY.png',
 '../data/MEX/484005004/pngs/484005004_2010_all_box484005004_MAY.png',
 '../data/MEX/484005008/pngs/484005008_2010_all_box484005008_MAY.png',
 '../data/MEX/484001002/pngs/484001002_2010_all_box484001002_MAY.png',
 '../data/MEX/484004006/pngs/484004006_2010_all_box484004006_MAY.png',
 '../d

In [9]:
butler.train_attn_model(train_dl, val_dl)

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch:  0
  Training Loss:  2169.4016865412395
  Validation Loss:  1376.1240332126617


Epoch:  1
  Training Loss:  2141.4638673782347
  Validation Loss:  1370.288335418701


Epoch:  2
  Training Loss:  2120.110971705119
  Validation Loss:  1353.5679107666015


Epoch:  3
  Training Loss:  2100.273968887329
  Validation Loss:  1349.6906396865845


Epoch:  4
  Training Loss:  2081.2946067810058
  Validation Loss:  1316.7540157318115
Percentage above threshold:  1.0
  Moving to threshold:  8


Epoch:  5
  Training Loss:  2065.893147532145
  Validation Loss:  1300.5041847229004


Epoch:  6
  Training Loss:  2047.6519706726074
  Validation Loss:  1297.1062446594237


Epoch:  7
  Training Loss:  2034.3321756998698
  Validation Loss:  1280.8518135070801


Epoch:  8
  Training Loss:  2025.7687525431315
  Validation Loss:  1289.7690597534179
Percentage above threshold:  1.0
  Moving to threshold:  7


Epoch:  9
  Training Loss:  2017.6233861287435
  Validation Loss:  1283.9876899719238


Epoch:

In [15]:
butler.train_fc_model(train_dl, val_dl, outside_estimator = 'mlp')

MLP MAE:  5141.210155393997


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [10]:
def get_y(json_path, estimates):
    
    """
    Get a list of the y features corresponding to each image in image_names
    """
    
    m = open(json_path,)
    mig_data = json.load(m)
    m.close()
    muni_ids = [i for i in estimates]
    print(len(muni_ids), "municipalities.")  
    return [mig_data[i] for i in muni_ids]

In [11]:
x = np.array(list(butler.scale_estimates.values()))
y = np.array(get_y("../../pooling/data/migration_data.json", butler.scale_estimates.keys()))

15 municipalities.


In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

In [23]:
def mae(true, pred):
    diff = np.abs(true - pred)
    return np.mean(diff)

In [24]:
rfr = RandomForestRegressor()
rfr.fit(x, y)
mae(y, rfr.predict(x))

2712.458666666667

In [31]:
dtr = DecisionTreeRegressor(max_depth = 5)
dtr.fit(x, y)
mae(y, dtr.predict(x))

7.933333333333334

In [36]:
mlp = MLPRegressor()
mlp.fit(x, y)
mae(y, mlp.predict(x))

/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


5096.862278985803

In [38]:
knn = KNeighborsRegressor()
knn.fit(x, y)
mae(y, knn.predict(x))

4149.666666666666